### Build the dataframe from the raw files

In [1]:
# Import required packages
import pandas as pd
import os
import xml.etree.ElementTree as ET
import spacy
import re
from lingua import Language, LanguageDetectorBuilder

In [2]:
def parse_xml(xml_file):
    global empty_counter

    # Load the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Define the namespace
    namespace = {
        'atom': 'http://www.w3.org/2005/Atom',
        'nitf': 'http://iptc.org/std/NITF/2006-10-18/',
        'dc': 'http://purl.org/dc/elements/1.1/'
    }

    # Extract the ID
    nexis_id = root.find('.//atom:id', namespace).text
    nexis_id = nexis_id.split('Item:')[1]
    nexis_id = nexis_id.replace("-", "")

    # Extract the title
    title = root.find('.//atom:title', namespace).text

    # Extract the published date
    published_date = root.find('.//atom:published', namespace).text
    published_date = published_date.split('T')[0]

    # Extract the publisher
    publisher = root.find('.//metadata/publicationInfo/publicationName').text


    # Extract the body text
    body_text = root.find('.//nitf:body.content//bodyText//p', namespace)

    # Extract all body paragraphs and join them into a single line
    body_paragraphs = []
    body_text_element = root.find('.//nitf:body.content//bodyText', namespace)
    if body_text_element is not None:
        for p_element in body_text_element.findall('.//p', namespace):
            if p_element.text:
                body_paragraphs.append(p_element.text.strip())
    text = ' '.join(body_paragraphs)

    # Check for empty text bodies and return
    if text == '':
        #print('No text body in file ' + file_path)
        empty_counter += 1
        return

    # Create a dictionary with the extracted values
    data_dict = {
        'nexis_id': [nexis_id],
        'title': [title],
        'publication_date': [published_date],
        'publisher': [publisher],
        'text': [text]
    }

    df = pd.DataFrame(data_dict)
    return df

In [3]:
# Root Directory of the data
folder_path = 'c:\\Friendcloud\\_University\\_Masterarbeit\\Data\\Nexis\\'

# Initialize empty counter
empty_counter = 0

# Go through all subdirectories and fill raw_data with the data
raw_data = []
for root, _, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.xml'):
            try:
                file_path = os.path.join(root, file)
                data = parse_xml(file_path)
                raw_data.append(data)
            except:
                print('Error in file ' + file_path)

# Build pandas DataFrame
df_raw = pd.concat(raw_data, ignore_index=True)
print('Number of articles removed: ' + str(empty_counter))
print('Number of articles accepted: ' + str(len(df_raw.index)))

Error in file c:\Friendcloud\_University\_Masterarbeit\Data\Nexis\WebnewsGerman\68SBV9Y1JCSCJ3HV0000000_Christine_Figgener_im_KlimaLabor_Mee.xml
Number of articles removed: 90
Number of articles accepted: 6778


In [ ]:
df_raw

In [ ]:
df_raw.to_pickle("../Data/df_raw.pkl")

### Basic Data preprocessing

In [5]:
df_processed = pd.read_pickle("../Data/df_raw.pkl")

In [6]:
# Add year of publication to dataset
df_processed['publication_date'] = pd.to_datetime(df_processed['publication_date'], errors='coerce')
df_processed['publication_year'] = df_processed['publication_date'].dt.year
df_processed = df_processed[['nexis_id', 'title', 'publication_date', 'publication_year', 'publisher', 'text']]

In [7]:
# Remove articles with duplicate id
count_before = len(df_processed.index)
df_processed = df_processed.drop_duplicates(subset='nexis_id', keep='first')
count_after = len(df_processed.index)
print('Number of articles removed: ' + str(count_before - count_after))

Number of articles removed: 3


In [8]:
# Find examples of articles with duplicate text
df_processed[df_processed.duplicated(subset='text', keep=False)].sample(10)

,nexis_id,title,publication_date,publication_year,publisher,text
169,5WS5DD21JCXB124H0000000,"""Clean Meat"" - Der Burger kommt bald aus dem L...",2019-07-14,2019,AWP Premium Swiss News (German),Madrid/Zürich (awp/awp/sda/reu) - Als das nied...
3930,5W04RGF1F16T238K0000000,Experten für den Weltuntergang,2019-04-27,2019,Neue Zürcher Zeitung (Internationale Ausgabe) ...,Grün ist die Trendfarbe der politischen Modesa...
36,63MDR2S1JBN970T50000000,Bühler & Co. tüfteln an Fleisch aus dem Labor,2021-09-17,2021,Appenzeller Zeitung,Zusammen mit der Migros und Givaudan will der ...
4136,59G7CTX1DYJ2Y2CN0000000,Hat Kohl den Mut? Vielfach manipuliert Erkundu...,1994-06-16,1994,Nürnberger Nachrichten,Auch in Bonn? Kürzlich sind Äußerungen des Bun...
6498,5SWJVDC1F16N03330000000,Schnitt ins Erbgut,2018-07-25,2018,WELT ONLINE (Deutsch),"Der Europäische Gerichtshof hat es abgelehnt, ..."
2265,4CC11S80TWH7K2TP0000000,Verbraucherschutz büßt 460 000 Euro ein,2004-04-27,2004,Frankfurter Rundschau,"VON FRANK SCHUMANN Wiesbaden · 26. April · 1,..."
3726,66R6X6T1JBN8P3V90000000,Tierische Proteine einmal anders,2022-10-29,2022,Märkische Oderzeitung Frankfurter Stadtbote (O...,Eine Catering-Küche in Kopenhagen. Kessel und ...
6573,66K42H61F16N028M0000000,Wursthersteller,2022-10-08,2022,WELT ONLINE (Deutsch),Der niedersächsische Wursthersteller Rügenwald...
226,5WXCJC21JCXB14280000000,Algen und Laborfleisch: UBS sieht alternative ...,2019-07-18,2019,AWP Premium Swiss News (German),ZÜRICH (awp international) - Hersteller neuer ...
79,5WMHB9Y1F0CCR50R0000000,"""Clean Meat"" - Der Burger kommt bald aus dem L...",2019-07-14,2019,AWP Premium Swiss News (German),Madrid/Zürich (awp/awp/sda/reu) - Als das nied...


In [9]:
# Remove articles with duplicate text
count_before = len(df_processed.index)
df_processed = df_processed.drop_duplicates(subset='text', keep='first')
count_after = len(df_processed.index)
print('Number of articles removed: ' + str(count_before - count_after))

Number of articles removed: 1278


In [10]:
# Remove articles with length less than 100 characters
count_before = len(df_processed.index)
df_processed = df_processed[df_processed['text'].str.len() > 100]
count_after = len(df_processed.index)
print('Number of articles removed: ' + str(count_before - count_after))

Number of articles removed: 73


In [11]:
# Remove articles which are not in German using lingua
detector = LanguageDetectorBuilder.from_all_spoken_languages().build()

# Add empty language column to dataset
df_processed['language'] = ''

# Detect language of each article
for index, row in df_processed.iterrows():
    text = row['text']
    language = detector.detect_language_of(text)
    df_processed.at[index, 'language'] = language.iso_code_639_1.name

# Print the language and publisher of articles which are not in German
print(df_processed[df_processed['language'] != 'DE']['language'].value_counts())
print(df_processed[df_processed['language'] != 'DE']['publisher'].value_counts())

# Remove articles which are not in German
count_before = len(df_processed.index)
df_processed = df_processed[df_processed['language'] == 'DE']
count_after = len(df_processed.index)
print('\nNumber of articles removed: ' + str(count_before - count_after))

language
EN    118
ES      1
Name: count, dtype: int64
publisher
dpa-AFX ProFeed                                                              19
News Bites - People in Business                                              12
American Banking and Market News                                             10
Spiegel Online                                                                7
Industry SnapShot                                                             4
GJAE - German Journal of Agricultural Economics (ehemals Agrarwirtschaft)     4
Agrarwirtschaft                                                               4
Newstex Blogs                                                                 4
Industry SnapShot Summary                                                     3
PR Newswire                                                                   2
MENAFN - Market Reports (English)                                             2
MENAFN - Press Releases (English)                      

In [12]:
# Reindex the dataframe
df_processed = df_processed.reset_index(drop=True)

In [13]:
df_processed

,nexis_id,title,publication_date,publication_year,publisher,text,language
0,5MNMH621JB0GF09H0000000,Angst vor dem «harten Brexit» auf der Insel - ...,2017-01-15,2017,AP Deutsch,Wenn Großbritannien Ende März den Ausstieg aus...,DE
1,5SM3THB1DXFJ50MP0000000,Fleischindustrie wehrt sich gegen Marketing fü...,2018-06-21,2018,AP Deutsch,Fleisch aus dem Labor ist noch eine Zukunftsvi...,DE
2,5SM3THB1DXFJ50MY0000000,Der AP-Überblick am Nachmittag,2018-06-21,2018,AP Deutsch,Die AP Weltnachrichten haben heute unter ander...,DE
3,5STNVWH1DXFJ53VM0000000,Laborfleisch soll in drei Jahren auf die Telle...,2018-07-17,2018,AP Deutsch,Maastricht (AP) - Das niederländische Unterneh...,DE
4,5VHK2XG1JB0GF4Y50000000,Israelische Forscher wollen künstliche Steaks ...,2019-02-26,2019,AP Deutsch,"Die Weltbevölkerung wächst, die Nachfrage nach...",DE
...,...,...,...,...,...,...,...
5301,67KW1VK1F15WB4660000000,Kein Titel,2023-02-21,2023,ZEIT Wissen,6 Am anfang drei Fragen 1. Können wir andere m...,DE
5302,67KW1VK1F15WB46B0000000,Leben und schmecken lassen,2023-02-21,2023,ZEIT Wissen,"Ein saftiges Filet, für das kein Huhn sterben ...",DE
5303,7W29GN20YC2460S30000000,ABSCHIED VOM ALTEN ITALIEN,2009-05-13,2009,ZEIT Wissen,John Dickie: »Delizia! Die Italiener und ihre...,DE
5304,7X8DW4712SK2G0H10000000,Essen aus dem Labor,2009-12-08,2009,ZEIT Wissen,Es ist der letzte Tag auf der Lebensmittelmess...,DE


In [14]:
df_processed.to_pickle("../Data/df.pkl")

### Spacy

In [15]:
# Small spacy model
nlp = spacy.load("de_core_news_sm")
df_spacy_sm = pd.read_pickle("../Data/df.pkl")
df_spacy_sm["spacy_sm"] = df_spacy_sm["text"].apply(lambda x: nlp(x))
df_spacy_sm.to_pickle("../Data/df_spacy_sm.pkl")

### Text Clean up

In [16]:
VARIANT = "LARGE" # "SMALL", "MEDIUM" , "FULL"

if VARIANT == "SMALL":
    df_processed = pd.read_pickle("../Data/df.pkl")
    df_processed = df_processed.head(20)
elif VARIANT == "MEDIUM":
    df_processed = pd.read_pickle("../Data/df.pkl")
    df_processed = df_processed.head(500)
else:
    df_processed = pd.read_pickle("../Data/df.pkl")

In [17]:
df_processed

,nexis_id,title,publication_date,publication_year,publisher,text,language
0,5MNMH621JB0GF09H0000000,Angst vor dem «harten Brexit» auf der Insel - ...,2017-01-15,2017,AP Deutsch,Wenn Großbritannien Ende März den Ausstieg aus...,DE
1,5SM3THB1DXFJ50MP0000000,Fleischindustrie wehrt sich gegen Marketing fü...,2018-06-21,2018,AP Deutsch,Fleisch aus dem Labor ist noch eine Zukunftsvi...,DE
2,5SM3THB1DXFJ50MY0000000,Der AP-Überblick am Nachmittag,2018-06-21,2018,AP Deutsch,Die AP Weltnachrichten haben heute unter ander...,DE
3,5STNVWH1DXFJ53VM0000000,Laborfleisch soll in drei Jahren auf die Telle...,2018-07-17,2018,AP Deutsch,Maastricht (AP) - Das niederländische Unterneh...,DE
4,5VHK2XG1JB0GF4Y50000000,Israelische Forscher wollen künstliche Steaks ...,2019-02-26,2019,AP Deutsch,"Die Weltbevölkerung wächst, die Nachfrage nach...",DE
...,...,...,...,...,...,...,...
5301,67KW1VK1F15WB4660000000,Kein Titel,2023-02-21,2023,ZEIT Wissen,6 Am anfang drei Fragen 1. Können wir andere m...,DE
5302,67KW1VK1F15WB46B0000000,Leben und schmecken lassen,2023-02-21,2023,ZEIT Wissen,"Ein saftiges Filet, für das kein Huhn sterben ...",DE
5303,7W29GN20YC2460S30000000,ABSCHIED VOM ALTEN ITALIEN,2009-05-13,2009,ZEIT Wissen,John Dickie: »Delizia! Die Italiener und ihre...,DE
5304,7X8DW4712SK2G0H10000000,Essen aus dem Labor,2009-12-08,2009,ZEIT Wissen,Es ist der letzte Tag auf der Lebensmittelmess...,DE


In [18]:
# Load the German language model in Spacy
nlp = spacy.load('de_core_news_sm')

# Define a function to clean the text
def clean_text(text):
    # Remove double dashes
    text = text.replace('--', ' ')

    # Remove special characters
    text = re.sub(r"[^A-Za-z0-9äöüÄÖÜß ]+", '', text)
    
    # Lemmatize the text
    doc = nlp(text)
    lemmatized_text = ' '.join([token.lemma_.lower() for token in doc])

    # If the first word has the label "GPE", remove it
    if doc[0].ent_type_ == 'GPE':
        print('GPE found')
        lemmatized_text = lemmatized_text.split(' ', 1)[1]
        
    return lemmatized_text

# Apply the clean_text function to the 'text' column in df_processed and save as a new dataframe df_clean
df_clean = df_processed.copy()
df_clean['text'] = df_clean['text'].apply(clean_text)

In [19]:
df_clean

,nexis_id,title,publication_date,publication_year,publisher,text,language
0,5MNMH621JB0GF09H0000000,Angst vor dem «harten Brexit» auf der Insel - ...,2017-01-15,2017,AP Deutsch,wenn großbritannien ende märz der ausstieg aus...,DE
1,5SM3THB1DXFJ50MP0000000,Fleischindustrie wehrt sich gegen Marketing fü...,2018-06-21,2018,AP Deutsch,fleisch aus der labor sein noch ein zukunftsvi...,DE
2,5SM3THB1DXFJ50MY0000000,Der AP-Überblick am Nachmittag,2018-06-21,2018,AP Deutsch,der ap weltnachrichten haben heute unter ander...,DE
3,5STNVWH1DXFJ53VM0000000,Laborfleisch soll in drei Jahren auf die Telle...,2018-07-17,2018,AP Deutsch,maastricht ap der niederländisch unternehmen...,DE
4,5VHK2XG1JB0GF4Y50000000,Israelische Forscher wollen künstliche Steaks ...,2019-02-26,2019,AP Deutsch,der weltbevölkerung wachsen der nachfrage nach...,DE
...,...,...,...,...,...,...,...
5301,67KW1VK1F15WB4660000000,Kein Titel,2023-02-21,2023,ZEIT Wissen,6 an anfang drei frage 1 können wir anderer mi...,DE
5302,67KW1VK1F15WB46B0000000,Leben und schmecken lassen,2023-02-21,2023,ZEIT Wissen,ein saftig filet für der kein huhn sterben mus...,DE
5303,7W29GN20YC2460S30000000,ABSCHIED VOM ALTEN ITALIEN,2009-05-13,2009,ZEIT Wissen,john dickie delizia der italiener und ihr kü...,DE
5304,7X8DW4712SK2G0H10000000,Essen aus dem Labor,2009-12-08,2009,ZEIT Wissen,es sein der letzter tag auf der lebensmittelme...,DE


In [20]:
df_clean.to_pickle("../Data/df_clean.pkl")